# Sprint 3 - Machine Learning e Redes Neurais

Integrantes:

95985 - Henrico Nardelli Bela<br>
95044 - Guilherme Antonio Silva<br>
96232 - Felype Nunes de Souza<br>
94341 - Rafael Pereira da Silva<br>

#### To do!

Análise PREDITIVA dos dados de NPS  

A partir das bases disponibilizadas pela BASF, e do trabalho de limpeza realizado na Sprint #2, desenvolver uma análise PREDITIVA para o problema. Entregar um notebook com o pipeline implementado contendo: (1) leitura e limpeza dos dados (podem emprestar do trabalho já realizado na Sprint #2), (2) modelo preditivo e (3) resultados. Podem trabalhar com análise supervisionada ou não-supervisionada. Se for o caso, destaquem qual é seu label. Aqui vale detectar outliers, reduzir dimensionalidade (colunas), tratar dados desbalanceado, fazer eng de features, etc... Não se esqueçam de tecer análises críticas dos achados em comentários markdown.  

Pensem neste modelo como um v0 exploratório a ser apresentado para os diretores da BASF narrando os desafios / potenciais dos dados e do problema.  

=> Entregar um jupyter notebook, incluindo nomes e RMs do grupo.  

=> OBSERVAÇÃO: esta entrega será a mesma para Machine Learning & Redes. Caprichem! 

---

## 1º Importando libs

In [397]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.tree import export_graphviz

from subprocess import call
from IPython.display import Image

import warnings

warnings.filterwarnings("ignore")

## 2º Lendo base

In [341]:
df = pd.read_excel("basf_nps2019.xlsx")
df

,Survey ID,Contact Country,Job Title,Contact Department,Sold to,Lead Contact OD,Main Industry,Customer Network,Local Invitation Date,Survey status,...,Product Quality,Delivery Reliability,Understanding your business needs,Alert type,Current status of alert,Local Rapid Response Sent Date,Due Date 20d,Local Alert Closed Date,What was the result of the follow-up conversation?,Outerloop Activity
0,2335847,Brazil,Analyst,R&D,394809,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
1,2335846,Brazil,Director,Others,394809,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
2,2335845,Brazil,Analyst,Supply Chain,1043002,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
3,2335844,Brazil,Manager,Others,2501948,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,COMPLETED,...,10.0,7.0,10.0,Passive,Reminded,2022-04-08 16:39:25.287,2022-05-05 11:34:58,NaT,NaN,NaN
4,2335843,Brazil,Manager,Supply Chain,1323633,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,COMPLETED,...,10.0,10.0,10.0,Promoter,Closed,2022-04-08 16:35:54.100,2022-05-05 15:04:18,2022-05-02 10:37:50.286,No issues,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39576,136716,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,COMPLETED,...,10.0,9.0,8.0,Promoter,Closed,2019-03-20 13:16:36.086,2019-04-16 21:54:25,2019-04-11 10:22:54.168,No issues,NaN
39577,136715,Brazil,Purchaser,Procurement,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,COMPLETED,...,9.0,8.0,8.0,Promoter,Closed,2019-03-20 13:11:09.971,2019-04-16 11:07:38,2019-04-11 10:26:07.302,No issues,NaN
39578,136714,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
39579,136713,Brazil,Manager,Management,2049754,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN


## 3º Algumas analises exploratórias da base

In [342]:
df.columns

Index(['Survey ID', 'Contact Country', 'Job Title', 'Contact Department',
       'Sold to', 'Lead Contact OD', 'Main Industry', 'Customer Network',
       'Local Invitation Date', 'Survey status', 'Local Response Date',
       'Likelihood to Recommend', 'Ease of doing business', 'Issue Resolution',
       'Product Quality', 'Delivery Reliability',
       'Understanding your business needs', 'Alert type',
       'Current status of alert', 'Local Rapid Response Sent Date',
       'Due Date 20d', 'Local Alert Closed Date',
       'What was the result of the follow-up conversation?',
       'Outerloop Activity'],
      dtype='object')

In [343]:
df.shape

(39581, 24)

In [344]:
df.head(10)

,Survey ID,Contact Country,Job Title,Contact Department,Sold to,Lead Contact OD,Main Industry,Customer Network,Local Invitation Date,Survey status,...,Product Quality,Delivery Reliability,Understanding your business needs,Alert type,Current status of alert,Local Rapid Response Sent Date,Due Date 20d,Local Alert Closed Date,What was the result of the follow-up conversation?,Outerloop Activity
0,2335847,Brazil,Analyst,R&D,394809,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
1,2335846,Brazil,Director,Others,394809,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
2,2335845,Brazil,Analyst,Supply Chain,1043002,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
3,2335844,Brazil,Manager,Others,2501948,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,COMPLETED,...,10.0,7.0,10.0,Passive,Reminded,2022-04-08 16:39:25.287,2022-05-05 11:34:58,NaT,NaN,NaN
4,2335843,Brazil,Manager,Supply Chain,1323633,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,COMPLETED,...,10.0,10.0,10.0,Promoter,Closed,2022-04-08 16:35:54.100,2022-05-05 15:04:18,2022-05-02 10:37:50.286,No issues,NaN
5,2335842,Brazil,Director,Management,901687,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
6,2335841,Brazil,Director,Supply Chain,1965027,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
7,2335840,Brazil,Director,Others,1323633,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERY_BOUNCED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
8,2335838,Brazil,Manager,R&D,1043002,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
9,2335837,Brazil,Manager,Procurement,1043002,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN


In [345]:
df.tail(10)

,Survey ID,Contact Country,Job Title,Contact Department,Sold to,Lead Contact OD,Main Industry,Customer Network,Local Invitation Date,Survey status,...,Product Quality,Delivery Reliability,Understanding your business needs,Alert type,Current status of alert,Local Rapid Response Sent Date,Due Date 20d,Local Alert Closed Date,What was the result of the follow-up conversation?,Outerloop Activity
39571,136721,Brazil,Analyst,R&D,1568982,CM - Monomers,Others,No,2019-03-19 09:00:00,COMPLETED,...,10.0,9.0,9.0,Promoter,Closed,2019-03-19 15:50:02.915,2019-04-16 09:08:09,2021-09-23 15:06:24.412,Issue(s) identified and unresolved,Open
39572,136720,Brazil,Analyst,Supply Chain,1568982,CM - Monomers,Others,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
39573,136719,Brazil,Purchaser,Procurement,1049008,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
39574,136718,Brazil,Purchase,Procurement,1970601,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
39575,136717,Brazil,Purchaser,Procurement,1955698,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
39576,136716,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,COMPLETED,...,10.0,9.0,8.0,Promoter,Closed,2019-03-20 13:16:36.086,2019-04-16 21:54:25,2019-04-11 10:22:54.168,No issues,NaN
39577,136715,Brazil,Purchaser,Procurement,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,COMPLETED,...,9.0,8.0,8.0,Promoter,Closed,2019-03-20 13:11:09.971,2019-04-16 11:07:38,2019-04-11 10:26:07.302,No issues,NaN
39578,136714,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
39579,136713,Brazil,Manager,Management,2049754,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
39580,136712,Brazil,Analyst,Supply Chain,472436,CM - Monomers,Others,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN


In [346]:
df.sample(10)

,Survey ID,Contact Country,Job Title,Contact Department,Sold to,Lead Contact OD,Main Industry,Customer Network,Local Invitation Date,Survey status,...,Product Quality,Delivery Reliability,Understanding your business needs,Alert type,Current status of alert,Local Rapid Response Sent Date,Due Date 20d,Local Alert Closed Date,What was the result of the follow-up conversation?,Outerloop Activity
20877,1083149,Brazil,Proprietário(a),Procurement,1045481,EC - Coatings,Consumer Goods,No,2020-09-15 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
27536,829057,Brazil,Purchasing,Procurement,4949656,EV - Performance Chemicals,Transportation & Automotive,No,2020-02-11 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
5066,2026388,Brazil,Proprietário(a),Management,1048105,EC - Coatings,Consumer Goods,No,2021-10-19 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
26889,878038,Argentina,Manager,R&D,2829154,EM - Care Chemicals,Chemicals & Plastics,No,2020-03-17 09:00:00,INVITATION_RESENT,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
7052,1988110,Brazil,Coordinator,R&D,1680105,EM - Care Chemicals,Consumer Goods,No,2021-09-28 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
5927,2023884,Brazil,Production Manager,Production,1043270,EN - Nutrition & Health,Nutrition & Health,No,2021-10-19 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
34207,475680,Brazil,CONSULTOR,Marketing,4662095,AP - Agricultural Solutions,Agriculture,No,2019-09-10 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
14100,1358405,Mexico,Srita.,Others,950027,AP - Agricultural Solutions,Agriculture,No,2021-04-06 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
3412,2097596,Brazil,Buyer,Procurement,1044771,CI - Intermediates,Others,No,2021-11-23 10:00:00,INVITATION_RESENT,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
18417,1139156,Brazil,Planejamento,Production,1426963,AP - Agricultural Solutions,Agriculture,No,2020-10-13 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN


In [347]:
df.describe()

,Survey ID,Likelihood to Recommend,Ease of doing business,Issue Resolution,Product Quality,Delivery Reliability,Understanding your business needs
count,3.958100e+04,12901.000000,12415.000000,12482.000000,12487.000000,12476.000000,12467.000000
mean,1.212484e+06,8.988218,8.163512,8.312690,9.432129,8.424816,8.275207
std,6.327991e+05,1.454429,1.785954,1.783772,0.907558,1.829802,1.765054
min,1.367120e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.267810e+05,8.000000,7.000000,8.000000,9.000000,8.000000,8.000000
50%,1.089055e+06,9.000000,8.000000,9.000000,10.000000,9.000000,9.000000
75%,1.845154e+06,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
max,2.335847e+06,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [348]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Survey ID,39581.0,1.212484e+06,632799.096486,136712.0,626781.0,1089055.0,1845154.0,2335847.0
Likelihood to Recommend,12901.0,8.988218e+00,1.454429,0.0,8.0,9.0,10.0,10.0
Ease of doing business,12415.0,8.163512e+00,1.785954,0.0,7.0,8.0,10.0,10.0
Issue Resolution,12482.0,8.312690e+00,1.783772,0.0,8.0,9.0,10.0,10.0
Product Quality,12487.0,9.432129e+00,0.907558,0.0,9.0,10.0,10.0,10.0
Delivery Reliability,12476.0,8.424816e+00,1.829802,0.0,8.0,9.0,10.0,10.0
Understanding your business needs,12467.0,8.275207e+00,1.765054,0.0,8.0,9.0,10.0,10.0


In [349]:
df.nunique()

Survey ID                                             39581
Contact Country                                          27
Job Title                                              3337
Contact Department                                       24
Sold to                                                4312
Lead Contact OD                                          12
Main Industry                                             9
Customer Network                                          2
Local Invitation Date                                   188
Survey status                                             8
Local Response Date                                   12506
Likelihood to Recommend                                  11
Ease of doing business                                   11
Issue Resolution                                         11
Product Quality                                          11
Delivery Reliability                                     11
Understanding your business needs       

In [350]:
df["Alert type"].value_counts()  # nosso label

Promoter     9228
Passive      2658
Detractor     654
Name: Alert type, dtype: int64

In [351]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39581 entries, 0 to 39580
Data columns (total 24 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   Survey ID                                           39581 non-null  int64         
 1   Contact Country                                     39581 non-null  object        
 2   Job Title                                           39560 non-null  object        
 3   Contact Department                                  39566 non-null  object        
 4   Sold to                                             39581 non-null  object        
 5   Lead Contact OD                                     39580 non-null  object        
 6   Main Industry                                       38284 non-null  object        
 7   Customer Network                                    39581 non-null  object        
 8   Local 

In [352]:
df.isna().sum()

Survey ID                                                 0
Contact Country                                           0
Job Title                                                21
Contact Department                                       15
Sold to                                                   0
Lead Contact OD                                           1
Main Industry                                          1297
Customer Network                                          0
Local Invitation Date                                     0
Survey status                                             0
Local Response Date                                   27041
Likelihood to Recommend                               26680
Ease of doing business                                27166
Issue Resolution                                      27099
Product Quality                                       27094
Delivery Reliability                                  27105
Understanding your business needs       

In [353]:
df.isnull().sum()

Survey ID                                                 0
Contact Country                                           0
Job Title                                                21
Contact Department                                       15
Sold to                                                   0
Lead Contact OD                                           1
Main Industry                                          1297
Customer Network                                          0
Local Invitation Date                                     0
Survey status                                             0
Local Response Date                                   27041
Likelihood to Recommend                               26680
Ease of doing business                                27166
Issue Resolution                                      27099
Product Quality                                       27094
Delivery Reliability                                  27105
Understanding your business needs       

In [354]:
df

,Survey ID,Contact Country,Job Title,Contact Department,Sold to,Lead Contact OD,Main Industry,Customer Network,Local Invitation Date,Survey status,...,Product Quality,Delivery Reliability,Understanding your business needs,Alert type,Current status of alert,Local Rapid Response Sent Date,Due Date 20d,Local Alert Closed Date,What was the result of the follow-up conversation?,Outerloop Activity
0,2335847,Brazil,Analyst,R&D,394809,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
1,2335846,Brazil,Director,Others,394809,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
2,2335845,Brazil,Analyst,Supply Chain,1043002,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
3,2335844,Brazil,Manager,Others,2501948,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,COMPLETED,...,10.0,7.0,10.0,Passive,Reminded,2022-04-08 16:39:25.287,2022-05-05 11:34:58,NaT,NaN,NaN
4,2335843,Brazil,Manager,Supply Chain,1323633,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,COMPLETED,...,10.0,10.0,10.0,Promoter,Closed,2022-04-08 16:35:54.100,2022-05-05 15:04:18,2022-05-02 10:37:50.286,No issues,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39576,136716,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,COMPLETED,...,10.0,9.0,8.0,Promoter,Closed,2019-03-20 13:16:36.086,2019-04-16 21:54:25,2019-04-11 10:22:54.168,No issues,NaN
39577,136715,Brazil,Purchaser,Procurement,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,COMPLETED,...,9.0,8.0,8.0,Promoter,Closed,2019-03-20 13:11:09.971,2019-04-16 11:07:38,2019-04-11 10:26:07.302,No issues,NaN
39578,136714,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
39579,136713,Brazil,Manager,Management,2049754,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN


#### Após a analise exploratória de nossos dados, podemos identificar que há uma grande dispariedade entre os labels, como detratores, promotores e passivos, toda a base tambem conta com muitos dados faltantes, pode-se resolver mais a frente com a limpeza e o uso de bibliotecas prontas para tratar os dados nulos. 

## 4º Limpeza da base

In [355]:
df = df.rename(
    columns={
        "Survey ID": "id",
        "Contact Country": "country",
        "Job Title": "job",
        "Contact Department": "department",
        "Sold to": "sold",
        "Lead Contact OD": "lead_OD",
        "Main Industry": "industry",
        "Customer Network": "network",
        "Local Invitation Date": "local_inv_date",
        "Survey status": "survey_stats",
        "Local Response Date": "local_resp_date",
        "Likelihood to Recommend": "like_recommend",
        "Ease of doing business": "do_business",
        "Issue Resolution": "issue_resolve",
        "Product Quality": "product_quality",
        "Delivery Reliability": "deliver_reliab",
        "Understanding your business needs": "business_need",
        "Alert type": "alert_type",
        "Current status of alert": "curr_stats_alert",
        "Local Rapid Response Sent Date": "local_sent_date",
        "Due Date 20d": "date_20d",
        "Local Alert Closed Date": "loc_alert_close_date",
        "What was the result of the follow-up conversation?": "followup_result",
        "Outerloop Activity": "outerloop",
    }
)

df.columns

Index(['id', 'country', 'job', 'department', 'sold', 'lead_OD', 'industry',
       'network', 'local_inv_date', 'survey_stats', 'local_resp_date',
       'like_recommend', 'do_business', 'issue_resolve', 'product_quality',
       'deliver_reliab', 'business_need', 'alert_type', 'curr_stats_alert',
       'local_sent_date', 'date_20d', 'loc_alert_close_date',
       'followup_result', 'outerloop'],
      dtype='object')

### Esta celula foi utilizada anteriormente no sprint 2, porem, com os avanços dos estudos, prosseguiremos com outra abordagem

    for coluna in df.columns:
        
        na_count = df[coluna].isna().value_counts()
        
        moda = statistics.mode(df[coluna])
        
        if df[coluna].dtype == "float64":
            if moda == "NaN":
                if na_count == True:
                    pass
            else: 
                df[coluna] = df[coluna].fillna(value = moda)
            
        if df[coluna].dtype == "int64":
            if moda == "NaN":
                if na_count == True:
                    pass
            else: 
                df[coluna] = df[coluna].fillna(value = moda)
                
        if df[coluna].dtype == "O":
            if moda == "NaN":
                if na_count == True:
                    pass
            else: 
                df[coluna] = df[coluna].fillna(value = moda)

### Iremos trabalhar com Ordinal Encoder, Simple Imputer, Column Transformer, Pipeline, e RFC (Random Forest Classifier)

### Primeiro precisamos criar nossa celula de label!
#### Um feature eng. meio que misturado com a limpeza rsrs

In [356]:
df["alert_type"].value_counts()

Promoter     9228
Passive      2658
Detractor     654
Name: alert_type, dtype: int64

In [357]:
df["label"] = df["alert_type"].apply(
    lambda x: 0
    if x == "Detractor"
    else 1
    if x == "Passive"
    else 2
    if x == "Promoter"
    else np.nan
)

In [358]:
df.label.value_counts()

2.0    9228
1.0    2658
0.0     654
Name: label, dtype: int64

In [359]:
df

,id,country,job,department,sold,lead_OD,industry,network,local_inv_date,survey_stats,...,deliver_reliab,business_need,alert_type,curr_stats_alert,local_sent_date,date_20d,loc_alert_close_date,followup_result,outerloop,label
0,2335847,Brazil,Analyst,R&D,394809,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
1,2335846,Brazil,Director,Others,394809,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
2,2335845,Brazil,Analyst,Supply Chain,1043002,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
3,2335844,Brazil,Manager,Others,2501948,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,COMPLETED,...,7.0,10.0,Passive,Reminded,2022-04-08 16:39:25.287,2022-05-05 11:34:58,NaT,NaN,NaN,1.0
4,2335843,Brazil,Manager,Supply Chain,1323633,CM - Monomers,Chemicals & Plastics,No,2022-04-05 10:00:00,COMPLETED,...,10.0,10.0,Promoter,Closed,2022-04-08 16:35:54.100,2022-05-05 15:04:18,2022-05-02 10:37:50.286,No issues,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39576,136716,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,COMPLETED,...,9.0,8.0,Promoter,Closed,2019-03-20 13:16:36.086,2019-04-16 21:54:25,2019-04-11 10:22:54.168,No issues,NaN,2.0
39577,136715,Brazil,Purchaser,Procurement,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,COMPLETED,...,8.0,8.0,Promoter,Closed,2019-03-20 13:11:09.971,2019-04-16 11:07:38,2019-04-11 10:26:07.302,No issues,NaN,2.0
39578,136714,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
39579,136713,Brazil,Manager,Management,2049754,CM - Monomers,Consumer Goods,No,2019-03-19 09:00:00,DELIVERED_AND_REMINDED,...,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN


In [360]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39581 entries, 0 to 39580
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    39581 non-null  int64         
 1   country               39581 non-null  object        
 2   job                   39560 non-null  object        
 3   department            39566 non-null  object        
 4   sold                  39581 non-null  object        
 5   lead_OD               39580 non-null  object        
 6   industry              38284 non-null  object        
 7   network               39581 non-null  object        
 8   local_inv_date        39581 non-null  datetime64[ns]
 9   survey_stats          39581 non-null  object        
 10  local_resp_date       12540 non-null  datetime64[ns]
 11  like_recommend        12901 non-null  float64       
 12  do_business           12415 non-null  float64       
 13  issue_resolve   

### Removemos algumas colunas que não serão necessarias!

In [361]:
df["local_sent_date"].dtype

dtype('<M8[ns]')

In [362]:
# df = df.drop(columns = ["local_inv_date", "local_resp_date", "local_sent_date", "id", "outerloop"])

for column in df:
    if df[column].dtype == "<M8[ns]":
        df = df.drop(columns=[column])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39581 entries, 0 to 39580
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                39581 non-null  int64  
 1   country           39581 non-null  object 
 2   job               39560 non-null  object 
 3   department        39566 non-null  object 
 4   sold              39581 non-null  object 
 5   lead_OD           39580 non-null  object 
 6   industry          38284 non-null  object 
 7   network           39581 non-null  object 
 8   survey_stats      39581 non-null  object 
 9   like_recommend    12901 non-null  float64
 10  do_business       12415 non-null  float64
 11  issue_resolve     12482 non-null  float64
 12  product_quality   12487 non-null  float64
 13  deliver_reliab    12476 non-null  float64
 14  business_need     12467 non-null  float64
 15  alert_type        12540 non-null  object 
 16  curr_stats_alert  12540 non-null  object

In [363]:
df = df.drop(columns=["id", "outerloop", "alert_type"])

In [364]:
df

,country,job,department,sold,lead_OD,industry,network,survey_stats,like_recommend,do_business,issue_resolve,product_quality,deliver_reliab,business_need,curr_stats_alert,followup_result,label
0,Brazil,Analyst,R&D,394809,CM - Monomers,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brazil,Director,Others,394809,CM - Monomers,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brazil,Analyst,Supply Chain,1043002,CM - Monomers,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Brazil,Manager,Others,2501948,CM - Monomers,Chemicals & Plastics,No,COMPLETED,8.0,10.0,9.0,10.0,7.0,10.0,Reminded,NaN,1.0
4,Brazil,Manager,Supply Chain,1323633,CM - Monomers,Chemicals & Plastics,No,COMPLETED,10.0,10.0,10.0,10.0,10.0,10.0,Closed,No issues,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39576,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,COMPLETED,10.0,8.0,9.0,10.0,9.0,8.0,Closed,No issues,2.0
39577,Brazil,Purchaser,Procurement,1680070,CM - Monomers,Consumer Goods,No,COMPLETED,9.0,9.0,8.0,9.0,8.0,8.0,Closed,No issues,2.0
39578,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39579,Brazil,Manager,Management,2049754,CM - Monomers,Consumer Goods,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [365]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39581 entries, 0 to 39580
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   country           39581 non-null  object 
 1   job               39560 non-null  object 
 2   department        39566 non-null  object 
 3   sold              39581 non-null  object 
 4   lead_OD           39580 non-null  object 
 5   industry          38284 non-null  object 
 6   network           39581 non-null  object 
 7   survey_stats      39581 non-null  object 
 8   like_recommend    12901 non-null  float64
 9   do_business       12415 non-null  float64
 10  issue_resolve     12482 non-null  float64
 11  product_quality   12487 non-null  float64
 12  deliver_reliab    12476 non-null  float64
 13  business_need     12467 non-null  float64
 14  curr_stats_alert  12540 non-null  object 
 15  followup_result   12362 non-null  object 
 16  label             12540 non-null  float6

### Identificamos uma dificuldade em utilizar o pipeline de ML de primeira mão, mas com algumas pesquisas, podemos utiliza-lo sem problemas, e não foi necessário utilizar o dropna.

In [366]:
# df = df.dropna(how = "any")
df

,country,job,department,sold,lead_OD,industry,network,survey_stats,like_recommend,do_business,issue_resolve,product_quality,deliver_reliab,business_need,curr_stats_alert,followup_result,label
0,Brazil,Analyst,R&D,394809,CM - Monomers,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brazil,Director,Others,394809,CM - Monomers,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brazil,Analyst,Supply Chain,1043002,CM - Monomers,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Brazil,Manager,Others,2501948,CM - Monomers,Chemicals & Plastics,No,COMPLETED,8.0,10.0,9.0,10.0,7.0,10.0,Reminded,NaN,1.0
4,Brazil,Manager,Supply Chain,1323633,CM - Monomers,Chemicals & Plastics,No,COMPLETED,10.0,10.0,10.0,10.0,10.0,10.0,Closed,No issues,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39576,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,COMPLETED,10.0,8.0,9.0,10.0,9.0,8.0,Closed,No issues,2.0
39577,Brazil,Purchaser,Procurement,1680070,CM - Monomers,Consumer Goods,No,COMPLETED,9.0,9.0,8.0,9.0,8.0,8.0,Closed,No issues,2.0
39578,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39579,Brazil,Manager,Management,2049754,CM - Monomers,Consumer Goods,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Base "limpa" rsrs, agora bora brincar de ML!!!

## 5º Separando em Treino e teste

In [367]:
df

,country,job,department,sold,lead_OD,industry,network,survey_stats,like_recommend,do_business,issue_resolve,product_quality,deliver_reliab,business_need,curr_stats_alert,followup_result,label
0,Brazil,Analyst,R&D,394809,CM - Monomers,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brazil,Director,Others,394809,CM - Monomers,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brazil,Analyst,Supply Chain,1043002,CM - Monomers,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Brazil,Manager,Others,2501948,CM - Monomers,Chemicals & Plastics,No,COMPLETED,8.0,10.0,9.0,10.0,7.0,10.0,Reminded,NaN,1.0
4,Brazil,Manager,Supply Chain,1323633,CM - Monomers,Chemicals & Plastics,No,COMPLETED,10.0,10.0,10.0,10.0,10.0,10.0,Closed,No issues,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39576,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,COMPLETED,10.0,8.0,9.0,10.0,9.0,8.0,Closed,No issues,2.0
39577,Brazil,Purchaser,Procurement,1680070,CM - Monomers,Consumer Goods,No,COMPLETED,9.0,9.0,8.0,9.0,8.0,8.0,Closed,No issues,2.0
39578,Brazil,Manager,Management,1680070,CM - Monomers,Consumer Goods,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39579,Brazil,Manager,Management,2049754,CM - Monomers,Consumer Goods,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [368]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39581 entries, 0 to 39580
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   country           39581 non-null  object 
 1   job               39560 non-null  object 
 2   department        39566 non-null  object 
 3   sold              39581 non-null  object 
 4   lead_OD           39580 non-null  object 
 5   industry          38284 non-null  object 
 6   network           39581 non-null  object 
 7   survey_stats      39581 non-null  object 
 8   like_recommend    12901 non-null  float64
 9   do_business       12415 non-null  float64
 10  issue_resolve     12482 non-null  float64
 11  product_quality   12487 non-null  float64
 12  deliver_reliab    12476 non-null  float64
 13  business_need     12467 non-null  float64
 14  curr_stats_alert  12540 non-null  object 
 15  followup_result   12362 non-null  object 
 16  label             12540 non-null  float6

In [369]:
df.sample(5)

,country,job,department,sold,lead_OD,industry,network,survey_stats,like_recommend,do_business,issue_resolve,product_quality,deliver_reliab,business_need,curr_stats_alert,followup_result,label
34379,Brazil,Balconista,Management,1085793,EC - Coatings,Consumer Goods,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3090,Brazil,Presidente,Management,4686827,ED - Dispersions & Pigments,Chemicals & Plastics,No,DELIVERED_AND_REMINDED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6594,Argentina,Specialist,Procurement,2015819,EM - Care Chemicals,Consumer Goods,No,COMPLETED,7.0,8.0,6.0,10.0,7.0,8.0,Closed,Issue(s) identified and resolved,1.0
26060,Brazil,Vendedor,Others,1034143,AP - Agricultural Solutions,Agriculture,No,COMPLETED,10.0,9.0,9.0,9.0,8.0,10.0,Closed,No issues,2.0
37511,Brazil,Technical,R&D,1323321,EM - Care Chemicals,Consumer Goods,No,DELIVERY_BOUNCED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [370]:
X = df

In [371]:
X.columns

Index(['country', 'job', 'department', 'sold', 'lead_OD', 'industry',
       'network', 'survey_stats', 'like_recommend', 'do_business',
       'issue_resolve', 'product_quality', 'deliver_reliab', 'business_need',
       'curr_stats_alert', 'followup_result', 'label'],
      dtype='object')

#### Aqui criamos duas listas com os dados separados, categoricos e numericos, para serem utilizados no pipeline mais adiante

In [372]:
categorical_columns = []
numerical_columns = []

for column in X.columns:
    if X[column].dtype == "O":
        categorical_columns.append(column)
    elif X[column].dtype == "float64":
        numerical_columns.append(column)

print(
    f"Categorical Columns {categorical_columns}\n\nNumerical Columns {numerical_columns}"
)

Categorical Columns ['country', 'job', 'department', 'sold', 'lead_OD', 'industry', 'network', 'survey_stats', 'curr_stats_alert', 'followup_result']

Numerical Columns ['like_recommend', 'do_business', 'issue_resolve', 'product_quality', 'deliver_reliab', 'business_need', 'label']


#### Realizamos a tratativa dos dados faltantes nos labels, ja que dentro do pipeline, o Simple Imputer apenas alimenta os dados, e não os labels. Demos uma randomizada nos labels, para dificultar para nosso modelo (/mal >:D)

In [373]:
X = X[categorical_columns + numerical_columns]
y = df["label"].fillna(random.randint(0, 2))

#### Separação :)

In [374]:
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### Pipeline!

#### Utilizamos o PCA para reduzir a dimensionalidade do dataset, e desoverfittar o modelo rsrs, encontramos a melhor dimensão a partir da 5º dimensão, no caso, a melhor foi em 10 dimensões.

#### Tambem utilizamos o Ordinal Encoder para os dados categoricos, e o Simple Imputer para os dados numericos

#### O modelo escolhido dessa vez, foi o Random Forest Classifier!

In [389]:
n_components = 10
dtm_i2 = list(range(0, len(x_train.columns) - 2))

pca = PCA(n_components=n_components)

categorical_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1, encoded_missing_value=-1
)

numerical_pipe = SimpleImputer(strategy="mean")

preprocess1 = ColumnTransformer(
    [
        ("cat", categorical_encoder, categorical_columns),
        ("num", numerical_pipe, numerical_columns),
    ],
    verbose_feature_names_out=False,
)

preprocess2 = ColumnTransformer([("pca", pca, dtm_i2)])

model = Pipeline(
    [
        ("preprocess1", preprocess1),
        ("preprocess2", preprocess2),
        ("classifier", RandomForestClassifier(random_state=42)),
    ]
)

model.fit(x_train, y_train)

Pipeline(steps=[('preprocess1',
                 ColumnTransformer(transformers=[('cat',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['country', 'job',
                                                   'department', 'sold',
                                                   'lead_OD', 'industry',
                                                   'network', 'survey_stats',
                                                   'curr_stats_alert',
                                                   'followup_result']),
                                                 ('num', SimpleImputer(),
                                                  ['like_recommend',
                                                   'do_business',
                                                   'issue_resolve',
                                                   'product_quality',
                                                   'deliver_reliab',
                                                   'business_need', 'label'])],
                                   verbose_feature_names_out=False)),
                ('preprocess2',
                 ColumnTransformer(transformers=[('pca', PCA(n_components=10),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14])])),
                ('classifier', RandomForestClassifier(random_state=42))])

#### Visualizando nossa floresta

estimator = model.estimators_[5]

export_graphviz(estimator, out_file='tree.dot', 
                feature_names = "Features",
                class_names = ["Promoter", "Detractor", "Passive"],
                rounded = True, proportion = False, 
                precision = 2, filled = True)


call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

Image(filename = 'tree.png')

In [395]:
print(f"Model Score Accuracy: {model.score(x_test, y_test):.2%}")
print("-" * 30)

Model Score Accuracy: 95.43%
------------------------------


# Conlusão

#### De acordo com alguns testes, o melhor score obtido pelo modelo foi o de 95%, com uma dimensionalidade de 10 colunas utilizando o PCA. O modelo apresentou um overfitting, porém ao diminuirmos a dimensão das colunas com o PCA, o modelo foi melhorando. Talvez o overfitting possa vir da nossa base de dados que apresenta muitos problemas, porém, pudemos identificar que o modelo se adequou bem ao dataset.